In [85]:
import requests
import json
import io
from contextlib import redirect_stdout
import pandas as pd
from tqdm.auto import tqdm
from ipywidgets import widgets
from ipywidgets import Layout
from ipywidgets import IntProgress, Layout, HBox, Label
from ipywidgets import Button
from functions.widgets import *
from functions.api import *
from functions.config import *
from functions.utils import *
from functions.georoc_api import GeoRocAPI

In [93]:
class MyApp:
    def __init__(self):
        self.api = None
        self.api_key_text = widgets.Password(description='API Key:')
        self.save_button = widgets.Button(description="Speichern")
        self.save_button.on_click(self.save_api_key)
        display(self.api_key_text, self.save_button)

    def save_api_key(self, sender):
        api_key = self.api_key_text.value
        print("API Key from input:", api_key)
        self.api = GeoRocAPI(api_key)
        GeoRocAPI.api_key = api_key  # Hinzufügen dieser Zeile
        print("API-Schlüssel gespeichert und API-Verbindung hergestellt!")


my_app = MyApp()


Password(description='API Key:')

Button(description='Speichern', style=ButtonStyle())

API Key from input: VGltbVdpcnR6OlZHbHRiVmRwY25SNlgwUkpSMGxUWDBGUVNWOHhOamM0TWpjME16WTA=
API-Schlüssel gespeichert und API-Verbindung hergestellt!


In [94]:
print("API Key in MyApp instance:", my_app.api.api_key)

API Key in MyApp instance: VGltbVdpcnR6OlZHbHRiVmRwY25SNlgwUkpSMGxUWDBGUVNWOHhOamM0TWpjME16WTA=


In [88]:
# Specify the desired keys
keys = ['Sample_Num', 'unique_id', 'Batches', 'References', 'SampleName', 'Location_Names', 'Location_Types', 
        'Loc_Data','Elevation_Min', 'Elevation_Max', 'Land_Or_Sea', 'Rock_Types', 'Rock_Classes', 'Rock_Textures', 
        'Age_Min', 'Age_Max', 'Materials', 'Minerals', 'Inclusion_Types', 'Location_Num', 'Latitude', 'Longitude', 
        'Latitude_Min', 'Latitude_Max', 'Longitude_Min', 'Longitude_Max', 'Tectonic_Setting', 'Method', 'Comment', 
        'Institution', 'Item_Name', 'Item_Group', 'Standard_Names', 'Standard_Values', 'Values', 'Units']


# Erstellen Sie eine Checkbox für jeden Schlüssel
checkboxes = [widgets.Checkbox(value=False, description=key) for key in keys]

# Erstellen Sie ein Layout für die GridBox
grid_layout = widgets.Layout(grid_template_columns="repeat(3, 300px)")  # 3 Spalten, jede 300px breit

# Erstellen Sie eine GridBox, um alle Checkboxen zu halten
grid = widgets.GridBox(children=checkboxes, layout=grid_layout)

display(grid)

GridBox(children=(Checkbox(value=False, description='Sample_Num'), Checkbox(value=False, description='unique_i…

In [89]:
def check_api_connection():
    endpoint = "ping"
    response = api_query(endpoint)

    if response is not None:
        print("Connection to API server successful!\n")
    else:
        print("Failed to connect to API server!\n")
        exit(1)

# Check the connection to the API server
check_api_connection()

API query successful for endpoint:ping 

Connection to API server successful!



In [90]:
limit_widget = widgets.Text(placeholder='Enter a value for Limit', description='Limit:')
offset_widget = widgets.Text(placeholder='Enter a value for Offset', description='Offset:')
location1_widget = widgets.Text(placeholder='Enter a value for Location1', description='Location1:')
location2_widget = widgets.Text(placeholder='Enter a value for Location2', description='Location2:')
location3_widget = widgets.Text(placeholder='Enter a value for Location3', description='Location3:')
setting_widget = widgets.Text(placeholder='Enter a value for Setting', description='Setting:')
latitude_widget = widgets.Text(placeholder='Enter a value for Latitude', description='Latitude:')
longitude_widget = widgets.Text(placeholder='Enter a value for Longitude', description='Longitude:')
rocktype_widget = widgets.Text(placeholder='Enter a value for Rocktype', description='Rocktype:')
rockclass_widget = widgets.Text(placeholder='Enter a value for Rockclass', description='Rockclass:')
mineral_widget = widgets.Text(placeholder='Enter a value for Mineral', description='Mineral:')
material_widget = widgets.Text(placeholder='Enter a value for Material', description='Material:')
inclusiontype_widget = widgets.Text(placeholder='Enter a value for Inclusiontype', description='Inclusiontype:')
sampletech_widget = widgets.Text(placeholder='Enter a value for SampleTech', description='SampleTech:')
element_widget = widgets.Text(placeholder='Enter a value for Element', description='Element:')
elementtype_widget = widgets.Text(placeholder='Enter a value for Elementtype', description='Elementtype:')
value_widget = widgets.Text(placeholder='Enter a value for Value', description='Value:')
title_widget = widgets.Text(placeholder='Enter a value for Title', description='Title:')
publicationyear_widget = widgets.Text(placeholder='Enter a value for Year of Publication', description='Year of Publication:')
doi_widget = widgets.Text(placeholder='Enter a value for DOI', description='DOI:')
firstname_widget = widgets.Text(placeholder='Enter a value for First Name', description='First Name:')
lastname_widget = widgets.Text(placeholder='Enter a value for Lastname', description='Lastname:')
agemin_widget = widgets.Text(placeholder='Enter a value for Age Minimum', description='Age Minimum:')
agemax_widget = widgets.Text(placeholder='Enter a value for Age Maximum', description='Age Maximum:')
geoage_widget = widgets.Text(placeholder='Enter a value for Geo Age', description='Geo Age:')
geoageprefix_widget = widgets.Text(placeholder='Enter a value for Geo Age Prefix', description='Geo Age Prefix:')
lab_widget = widgets.Text(placeholder='Enter a value for Lab', description='Lab:')

# Erstellen der Widget-Liste
widget_list = [
    limit_widget, offset_widget, location1_widget, location2_widget, location3_widget,
    setting_widget, latitude_widget, longitude_widget, rocktype_widget, rockclass_widget,
    mineral_widget, material_widget, inclusiontype_widget, sampletech_widget, element_widget,
    elementtype_widget, value_widget, title_widget, publicationyear_widget, doi_widget,
    firstname_widget, lastname_widget, agemin_widget, agemax_widget, geoage_widget,
    geoageprefix_widget, lab_widget
]

# Erstellen Sie ein Layout für die GridBox
grid_layout = widgets.Layout(grid_template_columns="repeat(3, 300px)")

# Erstellen Sie eine GridBox, um alle Widgets zu halten
grid2 = widgets.GridBox(children=widget_list, layout=grid_layout)

display(grid2)

GridBox(children=(Text(value='', description='Limit:', placeholder='Enter a value for Limit'), Text(value='', …

In [95]:
print(my_app.api)

filtered_samples_combined = my_app.api.get_filtered_samples(
    limit=limit_widget.value, 
    offset=offset_widget.value, 
    location1=location1_widget.value,
    location2=location2_widget.value,
    location3=location3_widget.value,
    setting=setting_widget.value,
    latitude=latitude_widget.value,
    longitude=longitude_widget.value,
    rocktype=rocktype_widget.value,
    rockclass=rockclass_widget.value,
    mineral=mineral_widget.value,
    material=material_widget.value,
    inclusiontype=inclusiontype_widget.value,
    sampletech=sampletech_widget.value,
    element=element_widget.value,
    elementtype=elementtype_widget.value,
    value=value_widget.value,
    title=title_widget.value,
    publicationyear=publicationyear_widget.value,
    doi=doi_widget.value,
    firstname=firstname_widget.value,
    lastname=lastname_widget.value,
    agemin=agemin_widget.value,
    agemax=agemax_widget.value,
    geoage=geoage_widget.value,
    geoageprefix=geoageprefix_widget.value,
    lab=lab_widget.value
)

print(filtered_samples_combined, "\n")

API query successful for endpoint:queries/samples 

{'NumItems': 2, 'Data': [{'SampleID': 0}, {'SampleID': 233338}]} 



In [96]:
# Extract SamplingFeatureIDs
if "Data" in filtered_samples_combined and filtered_samples_combined["Data"]:
    sampling_feature_ids = [sample["SampleID"] for sample in filtered_samples_combined["Data"]]
    print("\n", f"The extracted SampleIDs are:", sampling_feature_ids, "\n")
else:
    print("No data found or unexpected data structure", "\n")


 The extracted SampleIDs are: [0, 233338] 



In [97]:
# Create an empty DataFrame to store all measurement data
measurement_data = pd.DataFrame()

In [99]:
def get_measurement_data(api_key, sampling_feature_id):
    base_url = f"https://api-test.georoc.eu/api/v1/queries/fulldata/{sampling_feature_id}"
    headers = {
        "accept": "application/json",
        "DIGIS-API-ACCESSKEY": api_key
    }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        data = json.loads(response.text)['Data'][0]

        # Create a dictionary containing all keys and values from data
        df_data = {key: data[key] for key in keys}

        # Convert all values in df_data to lists
        for key, value in df_data.items():
            if not isinstance(value, list):
                df_data[key] = [value]

        # Find the longest list in df_data
        max_len = max([len(value) for value in df_data.values()])

        # Extend all the lists to the length of the longest list
        for key, value in df_data.items():
            if len(value) < max_len:
                df_data[key] = value + [None] * (max_len - len(value))

        # Create the DataFrame
        df = pd.DataFrame(df_data)

        return df

    else:
        print(f"Error fetching data for sample ID {sampling_feature_id}, Status code: {response.status_code}")
        return None

In [101]:
# Erstelle die Progressbar
progress = IntProgress(value=0, min=0, max=len(sampling_feature_ids), layout=Layout(width="auto"))
progress_label = Label('Starting...')
progress_box = HBox([progress, progress_label])
display(progress_box)

# Iterate over the list of SamplingFeatureIDs
for sampling_feature_id in sampling_feature_ids:
    print(f"Fetching measurement data for SamplingFeatureID: {sampling_feature_id}")

    # Get the measurement data for the current SamplingFeatureID
    df = get_measurement_data(my_app.api.api_key, sampling_feature_id)

    # Check if the dataframe is not empty and not None
    if df is not None and not df.empty:
        # Append the dataframe to the measurement_data DataFrame
        measurement_data = measurement_data.append(df, ignore_index=True)

        # Print the dataframe
        print(f"Data for SamplingFeatureID {sampling_feature_id}:\n", df)
    elif df is None:
        print(f"Error occurred while fetching data for SamplingFeatureID {sampling_feature_id}")
    else:
        print(f"No measurement data found for SamplingFeatureID {sampling_feature_id}")

    # Aktualisiere die Progressbar und das Label
    progress.value += 1
    progress_label.value = f"Processing: {sampling_feature_id} ({progress.value}/{progress.max})"

Fetching measurement data for SamplingFeatureID: 0
Error fetching data for sample ID 0, Status code: 404
Error occurred while fetching data for SamplingFeatureID 0
Fetching measurement data for SamplingFeatureID: 233338
Data for SamplingFeatureID 233338:
     Sample_Num unique_id   Batches  \
0     233338.0            880875.0   
1          NaN      None       NaN   
2          NaN      None       NaN   
3          NaN      None       NaN   
4          NaN      None       NaN   
5          NaN      None       NaN   
6          NaN      None       NaN   
7          NaN      None       NaN   
8          NaN      None       NaN   
9          NaN      None       NaN   
10         NaN      None       NaN   
11         NaN      None       NaN   
12         NaN      None       NaN   
13         NaN      None       NaN   
14         NaN      None       NaN   
15         NaN      None       NaN   
16         NaN      None       NaN   
17         NaN      None       NaN   
18         NaN      No

/tmp/ipykernel_63790/930552407.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  measurement_data = measurement_data.append(df, ignore_index=True)


In [ ]:
# Save the combined measurement_data DataFrame
measurement_data.to_csv('measurement_data.csv', index=False)